## "Схожесть" между двумя товарами

1. Необходимо построить эмбеддинги товаров и вычислить расстояние между ними (Сделано в задаче [Middle/SKU_Embeddings]())
2. Вычислим цены на похожие товары

Применимо в *динамическом ценообразовании*, *прогнозе спроса*

#### На входе два словаря:
- **embeddings** (словарь: идентификатор товарова - эмбеддинг)
- **prices** (словарь: идентификатор товарова - цена)

#### Класс SimilarItems с 4 методами:
- **similarity** (считает попарные похожести между всеми эмбеддингами, возвращая словарь сходств)
- **knn** (принимает результат работы функции similarity, и параметр top - кол-во ближайших соседей. Она выдает словарь с парами item_id - список top ближайших товаров)
- **knn_price** (принимает результат работы функции knn и словарь price. На выходе выдавая средневзвешенную цену top ближайших соседей)
- **transform** (Преобразует исходный словарь эмбеддингов в словарь с новыми ценами для всех товаров)

Ссылки:
- [Комбинации в Python](https://www.geeksforgeeks.org/python-itertools-combinations-function/)
- https://www.geeksforgeeks.org/python-sort-python-dictionaries-by-key-or-value/
- https://github.com/eugeneyan/applied-ml#embeddings

In [4]:
"""Solution for Similar Items task"""
from typing import Dict
from typing import List
from typing import Tuple

import numpy as np
from scipy.spatial.distance import cosine
from itertools import combinations


class SimilarItems:
    """Similar items class"""

    @staticmethod
    def similarity(embeddings: Dict[int, np.ndarray]) -> Dict[Tuple[int, int], float]:
        """Calculate pairwise similarities between each item
        in embedding.

        Args:
            embeddings (Dict[int, np.ndarray]): Items embeddings.

        Returns:
            Tuple[List[str], Dict[Tuple[int, int], float]]:
            List of all items + Pairwise similarities dict
            Keys are in form of (i, j) - combinations pairs of item_ids
            with i < j.
            Round each value to 8 decimal places.
        """
        
        pair_sims = {}
        emb_keys = list(embeddings.keys())
        emb_keys.sort()
        emb_sort = {key: embeddings[key] for key in emb_keys}
        comb = combinations(emb_sort, 2)
        for id1, id2 in comb:
            cos_sim = round(cosine(emb_sort[id1], emb_sort[id2]), 8)
            pair_sims[(id1, id2)] = cos_sim  
        
        return pair_sims

    @staticmethod
    def knn(
        sim: Dict[Tuple[int, int], float], top: int
    ) -> Dict[int, List[Tuple[int, float]]]:
        """Return closest neighbors for each item.

        Args:
            sim (Dict[Tuple[int, int], float]): <similarity> method output.
            top (int): Number of top neighbors to consider.

        Returns:
            Dict[int, List[Tuple[int, float]]]: Dict with top closest neighbors
            for each item.
        """
        #Можно иначе: 
        #ids = set(sum(tuple(sim.keys()), ()))
        ids1 = {x[0] for x in sim.keys()}
        ids2 = {x[1] for x in sim.keys()}
        ids = ids1.union(ids2)
    
        knn_dict = {}
        for i in ids:
            #Следующие 4 строчки можно заменить одним генератором словаря:
            #i_list = {key[0]+key[1]-i: value for key, value in sim.items() if i in key}
            i_list = []
            for keys, value in sim.items():
                if i in keys:
                    i_list.append((keys[0]+keys[1]-i, value))
            knn_dict[i] = sorted(i_list, key=lambda tup: tup[1], reverse=True)[:top]
            
        return knn_dict

    @staticmethod
    def knn_price(
        knn_dict: Dict[int, List[Tuple[int, float]]],
        prices: Dict[int, float],
    ) -> Dict[int, float]:
        """Calculate weighted average prices for each item.
        Weights should be positive numbers in [0, 2] interval.

        Args:
            knn_dict (Dict[int, List[Tuple[int, float]]]): <knn> method output.
            prices (Dict[int, float]): Price dict for each item.

        Returns:
            Dict[int, float]: New prices dict, rounded to 2 decimal places.
        """
        knn_price_dict = {}
        for key, value in knn_dict.items():
            weight_price = 0
            weight = 0
            for item in value:
                weight_price+=prices[item[0]]*(item[1]+1)
                weight+=(item[1]+1)
            knn_price_dict[key] = round(weight_price/weight, 2)
        return knn_price_dict

    @staticmethod
    def transform(
        embeddings: Dict[int, np.ndarray],
        prices: Dict[int, float],
        top: int,
    ) -> Dict[int, float]:
        """Transforming input embeddings into a dictionary
        with weighted average prices for each item.

        Args:
            embeddings (Dict[int, np.ndarray]): Items embeddings.
            prices (Dict[int, float]): Price dict for each item.
            top (int): Number of top neighbors to consider.

        Returns:
            Dict[int, float]: Dict with weighted average prices for each item.
        """
        pair_sims = SimilarItems.similarity(embeddings)
        knn_dict = SimilarItems.knn(pair_sims, top)
        knn_price_dict = SimilarItems.knn_price(knn_dict, prices)
        return knn_price_dict



In [5]:
#Sample data
embeddings = {
    2: np.array([-55.98, 82.87, 86.07, 18.71, -18.66, -46.74, -68.18, 60.29, 98.92, -78.95]),
    1: np.array([-26.57, -76.61, 81.61, -9.11, 74.8, 54.23, 32.56, -22.62, -72.44, -82.78]),
    3: np.array([-27.97, 25.39, -96.85, 3.51, 95.57, -27.48, -80.27, 8.39, 89.96, -36.68]),
    4: np.array([-37.0, -49.39, 43.3, 73.36, 29.98, -56.44, -15.91, -56.46, 24.54, 12.43]),
    5: np.array([-22.71, 4.47, -65.42, 10.11, 98.34, 17.96, -10.77, 2.5, -26.55, 69.16])
}


prices = {
    1: 100.5,
    2: 12.2,
    3: 60.0,
    4: 11.1,
    5: 245.2
}

In [6]:
si = SimilarItems()
si.transform(embeddings,prices,2)

{1: 36.73, 2: 177.52, 3: 59.84, 4: 154.29, 5: 52.29}